# Word Embedding

It is a word embedding project which analyses technology news to create a network by learning from the text content.

The dataset used in this project is from https://hakan.io/makine-ogrenmesi-turkce-haber-metinleri-veri-seti/

### Data

In [68]:
stopwords = [i.strip() for i in open("data/stopwords.txt", "r").readlines()]

In [69]:
import pandas as pd

data = pd.DataFrame(open("data/news.txt", "r").readlines(), columns =["News"])
data.head()

,News
0,"'Sürücüsüz otomobil yola çıktı', 'Modifiye oto..."
1,'Akıllı telefon bağımlılığına karşı \'Vekil Te...
2,'iPhone X ile iPhone 8 Plus karşı karşıya: Dah...
3,'Avrupa devleri hibrit elektrikli uçak için ko...
4,'11 yaşındaki kız 25 bin dolar değerindeki bil...


##### Remove punctuations

In [70]:
data["News"] = data["News"].str.replace('[^\w\s]', '')
data.head()

,News
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...
1,Akıllı telefon bağımlılığına karşı Vekil Telef...
2,iPhone X ile iPhone 8 Plus karşı karşıya Daha ...
3,Avrupa devleri hibrit elektrikli uçak için kol...
4,11 yaşındaki kız 25 bin dolar değerindeki bili...


##### Remove stopwords

In [71]:
data["News"] = data["News"].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
data.head()

,News
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...
1,Akıllı telefon bağımlılığına karşı Vekil Telef...
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...
3,Avrupa devleri hibrit elektrikli uçak kolları ...
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...


##### Features

In [72]:
import re

In [73]:
# Character count
data["Char"] = data["News"].apply(lambda x: len(re.sub(r"\s+", "", x)))
data.head()

,News,Char
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596
1,Akıllı telefon bağımlılığına karşı Vekil Telef...,802
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...,529
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544


In [74]:
# String count
data["Str"] = data["News"].apply(lambda x: len(re.sub(r"[^a-zA-ZığüşöçİĞÜŞÖÇ]+", "", x)))
data.head()

,News,Char,Str
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570
1,Akıllı telefon bağımlılığına karşı Vekil Telef...,802,801
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...,529,497
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536


In [75]:
# Integer count
data["Int"] = data["News"].apply(lambda x: len(re.sub(r"[^\d]+", "", x)))
data.head()

,News,Char,Str,Int
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570,26
1,Akıllı telefon bağımlılığına karşı Vekil Telef...,802,801,1
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...,529,497,32
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605,19
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536,8


In [76]:
# Word count
data["Word"] = data["News"].apply(lambda x: len(x.split()))
data.head()

,News,Char,Str,Int,Word
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570,26,85
1,Akıllı telefon bağımlılığına karşı Vekil Telef...,802,801,1,97
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...,529,497,32,101
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605,19,90
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536,8,77


In [78]:
# Average Word Length
data["Avg"] = data["News"].apply(lambda x: len(re.sub(r"\s+", "", x)) / len(x.split()))
data.head()

,News,Char,Str,Int,Word,Avg
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570,26,85,7.011765
1,Akıllı telefon bağımlılığına karşı Vekil Telef...,802,801,1,97,8.268041
2,iPhone X iPhone 8 Plus karşı karşıya Daha küçü...,529,497,32,101,5.237624
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605,19,90,6.933333
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536,8,77,7.064935


##### Frequent words

In [81]:
freq = pd.Series(" ".join(data["News"]).split()).value_counts()[:10]
freq

yeni         49
iPhone       41
r            32
2            28
telefon      24
Apple        23
teknoloji    23
sahip        23
ilk          22
8            22
dtype: int64

In [82]:
# Remove frequent words

data["News"] = data["News"].apply(lambda x: " ".join(x for x in x.split() if x not in list(freq.index)))
data.head()

,News,Char,Str,Int,Word,Avg
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570,26,85,7.011765
1,Akıllı bağımlılığına karşı Vekil Telefon Avust...,802,801,1,97,8.268041
2,X Plus karşı karşıya Daha küçük kasa büyük ekr...,529,497,32,101,5.237624
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605,19,90,6.933333
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536,8,77,7.064935


##### Rare words

In [83]:
rare = pd.Series(" ".join(data["News"]).split()).value_counts()[-10:]
rare

teknolojisinde    1
Kremlinin         1
kardeşler         1
Çine              1
türüne            1
gelişmeleri       1
bıraktı           1
yılın             1
3ü                1
İspatı            1
dtype: int64

In [84]:
# Remove rare words

data["News"] = data["News"].apply(lambda x: " ".join(x for x in x.split() if x not in list(rare.index)))
data.head()

,News,Char,Str,Int,Word,Avg
0,Sürücüsüz otomobil yola çıktı Modifiye otomobi...,596,570,26,85,7.011765
1,Akıllı bağımlılığına karşı Vekil Telefon Avust...,802,801,1,97,8.268041
2,X Plus karşı karşıya Daha küçük kasa büyük ekr...,529,497,32,101,5.237624
3,Avrupa devleri hibrit elektrikli uçak kolları ...,624,605,19,90,6.933333
4,11 yaşındaki kız 25 dolar değerindeki bilim öd...,544,536,8,77,7.064935


### TODO